Bubble Sheet Grader

In [11]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to display an image (optional for debugging)
def show_image(image, title="Image"):
    plt.figure(figsize=(5, 5))
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.title(title)
    plt.axis('off')
    plt.show()

#binary conversion
def bin_conversion(image, threshold=127):
#takes in grayscale image to be thresholded
#threshold is used such that pixels below which are set to 0 (black), above which set to 255 (white)
#returns a binary image (0 and 255 pixel values)
    binary_image = np.zeros_like(image, dtype=np.uint8)
    binary_image[image >= threshold] = 255
    binary_image[image < threshold] = 0
    return binary_image

# Circle detection using Hough Transform
def circles_with_hough(image, dp=1.2, min_dist=20, param1=80, param2=30, min_radius=20, max_radius=40):
#this function detects circles using Hough Transform
#takes in input a grayscale image
#dp is an inverse ratio of the accumulator resolution to the image resolution
#min_dist is minimum dis between detected circle centers
#param1 is gradient value used in the edge detection (Canny threshold)
#param2 is accumulator threshold for circle detection
#min_radiusis minimum radius of detected circles
#max_radius is maximum radius of detected circles
# returns detected circles as a list of (x, y, radius)

#circles detetced using Hough Transform    
    circles = cv2.HoughCircles(
        image,
        cv2.HOUGH_GRADIENT,
        dp=dp,
        minDist=min_dist,
        param1=param1,
        param2=param2,
        minRadius=min_radius,
        maxRadius=max_radius
    )#to ensure circles are detected and convert to int
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        return circles
    else:
        return []

#circles extracted, checking if filled, and computing marks based on correct answers
def extract_and_analyze_circles(image, circles, num_rows, correct_answers, dark_pixel_threshold=0.6):
  #first grouping circles by rows based on vertical (y-coor) position    
    rows = [[] for _ in range(num_rows)]
    row_height = (max(c[1] for c in circles) - min(c[1] for c in circles)) / num_rows
    min_y = min(c[1] for c in circles)
 
  #sorting each row's circles by x-coor
    for (x, y, r) in circles:
        row_idx = int((y - min_y) / row_height)
        row_idx = min(row_idx, num_rows - 1)
        rows[row_idx].append((x, y, r))

    rows = [sorted(row, key=lambda c: c[0]) for row in rows]

    filled_options_per_row = [[] for _ in range(num_rows)]
    for row_idx, row in enumerate(rows):
        for circle_idx, (x, y, r) in enumerate(row):
            #circular region extracted
            mask = np.zeros_like(image, dtype=np.uint8)
            cv2.circle(mask, (x, y), r, 255, -1)
            #croping the circular region
            min_x, max_x = max(0, x - r), min(image.shape[1], x + r)
            min_y, max_y = max(0, y - r), min(image.shape[0], y + r)
            circle_region = image[min_y:max_y, min_x:max_x]
            #ensuring the mask matches the size of the cropped region
            mask_cropped = mask[min_y:max_y, min_x:max_x]
            #applying the mask to the region
            cropped_masked = cv2.bitwise_and(circle_region, circle_region, mask=mask_cropped)
            binary_circle = bin_conversion(cropped_masked, threshold=127)#to binary
            dark_pixels = np.sum(binary_circle == 0)#counting white pixels
            total_pixels = binary_circle.size
            dark_pixel_ratio = dark_pixels / total_pixels
            #dtermining if circle is filled
            is_filled = dark_pixel_ratio >= dark_pixel_threshold
            if is_filled:
                filled_options_per_row[row_idx].append(circle_idx + 1)

    marks = 0
    results = []
    for row_idx, options in enumerate(filled_options_per_row):
        correct_answer = correct_answers[row_idx]
        if len(options) == 0:
            result = "No Answer"
            score = 0
        elif len(options) > 1:
            result = "Multiple Filled"
            score = -1
        elif options[0] == correct_answer:
            result = "Correct"
            score = 1
        else:
            result = "Wrong"
            score = 0
        marks += score
        results.append((row_idx + 1, options, correct_answer, result, score))

    return results, marks

#GUI
def analyze_omr():
    try:
        #get correct answers 
        correct_answers = []
        for i in range(10):
            answer = correct_answer_fields[i].get()
            if not answer.isdigit() or int(answer) < 1 or int(answer) > 5:
                messagebox.showerror("Error", f"Invalid answer for Q{i + 1}. Enter a number between 1 and 5.")
                return
            correct_answers.append(int(answer))

        #get image 
        image_path = filedialog.askopenfilename(title="Select Bubble sheet image", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if not image_path:
            return

        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.medianBlur(gray, 5)
        circles = circles_with_hough(blurred, min_radius=22, max_radius=30)

        if len(circles) == 0:
            messagebox.showerror("Error", "No circles detected!")
            return

        results, marks = extract_and_analyze_circles(gray, circles, num_rows=10, correct_answers=correct_answers)

        #results
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, "Results:\n")
        result_text.insert(tk.END, "-" * 40 + "\n")
        for q_no, options, correct, result, score in results:
            result_text.insert(tk.END, f"Q{q_no}: Filled = {options}, Correct = {correct}, Result = {result}, Marks = {score}\n")
        result_text.insert(tk.END, f"\nTotal Marks: {marks}/10")

        #display image
        image = Image.open(image_path)
        image.thumbnail((300, 300))
        tk_image = ImageTk.PhotoImage(image)
        canvas.image = tk_image
        canvas.create_image(150, 150, image=tk_image)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Create GUI
root = tk.Tk()
root.title("Bubble Sheet Analyzer")
root.geometry("900x900")

# Input for correct answers
tk.Label(root, text="Enter Correct Answers for 10 Questions:").pack(pady=5)

# Fields for correct answers
correct_answer_fields = []
for i in range(10):
    frame = tk.Frame(root)
    frame.pack(pady=2)
    tk.Label(frame, text=f"Q{i + 1}:").pack(side=tk.LEFT, padx=5)
    field = tk.Entry(frame, width=5)
    field.pack(side=tk.LEFT)
    correct_answer_fields.append(field)

# upload image button
tk.Button(root, text="Upload bubble sheet", command=analyze_omr).pack(pady=10)

canvas = tk.Canvas(root, width=300, height=250, bg="white")
canvas.pack(pady=10)

#textual results
result_text = tk.Text(root, wrap=tk.WORD, height=15)
result_text.pack(pady=5)

root.mainloop()
